## **Reformatting Interpreter Observations**

    This script converts all DB files from TSPO to CSV files. They are then combined based on the table that they are in, resulting in three CSV files: Display Table, Event Table, and Polygon Table. It then reformats the Event Table file (separating out the loss/stable/growth probabilities for each epoch) and attaches to it the point information from the Polygon Table file. 

##### Importing Packages

In [1]:
import sqlite3
import pandas as pd
import os
import json
import re
import numpy as np

##### Define path to the folder that contains the .db files. 

In [2]:
input_folder = 'C:/Users/McKenzie/Documents/OSU_Masters/intPoints/intPointsDEMO3THESIS'

#### Converting DB to CSV and combines by table. 

##### Function: Converts DB to CSV and merges each table into its own individual file. 

In [3]:
def sqlite_to_csv(input_folder):
    # A dictionary to keep track of dataframes for each table across all databases
    combined_dataframes = {}
    db_count = 0  # Counter to track how many .db files were processed

    # Process each file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.db'):
            db_file = os.path.join(input_folder, filename)

            # Extract the epoch number from the filename using regex
            epoch_match = re.search(r'epoch(\d{4})', filename)
            epoch_number = epoch_match.group(1) if epoch_match else 'Unknown'

            try:
                # Connect to the SQLite database
                conn = sqlite3.connect(db_file)
                db_count += 1  # Increment the count for every processed .db file
                
                # Get the list of tables
                query = "SELECT name FROM sqlite_master WHERE type='table';"
                tables = pd.read_sql_query(query, conn)
                
                if tables.empty:
                    print(f"No tables found in {db_file}.")
                    continue
                
                # Loop through each table and store data in memory
                for table_name in tables['name']:
                    try:
                        # Read the table into a DataFrame
                        df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
                        
                        if df.empty:
                            print(f"Table {table_name} in {db_file} is empty, skipping.")
                            continue
                        
                        # Add the epoch number as a new column
                        df['epoch'] = epoch_number

                        # Append the DataFrame to the appropriate list in the dictionary
                        if table_name not in combined_dataframes:
                            combined_dataframes[table_name] = []
                        combined_dataframes[table_name].append(df)
                    except Exception as e:
                        print(f"Error processing table {table_name} in {db_file}: {e}")
                
                # Close the database connection
                conn.close()
            except Exception as e:
                print(f"Error connecting to database {db_file}: {e}")

    # Print the total number of .db files processed
    print(f"Total number of .db files processed: {db_count}")

    # Now, combine all dataframes for each table and save to single CSV files
    for table_name, dfs in combined_dataframes.items():
        if dfs:
            combined_df = pd.concat(dfs, ignore_index=True)
            combined_df.to_csv(f"{table_name}_combined.csv", index=False)
            print(f"Combined data for {table_name} has been saved to {table_name}_combined.csv.")


##### Applying the above function. 

In [4]:
# Convert all DB files in folder to CSV and combine by table. 
sqlite_to_csv(input_folder)

Total number of .db files processed: 61
Combined data for displayTable has been saved to displayTable_combined.csv.
Combined data for eventTable has been saved to eventTable_combined.csv.
Combined data for polygonTable has been saved to polygonTable_combined.csv.


#### Reformatting the event table file (contains interpreter observations).  

##### Combining the spectral progression and spectral capacity columns. 
    Initially we were calling this uncertainty factor "progression" but this was switched to "capacity" part way through. 

In [5]:
# Load the CSV file
df_etCombo = pd.read_csv('eventTable_combined.csv')

# Combine 'Spectal_Progression' into 'Spectral_Capacity'
df_etCombo['Spectral_Capacity'] = df_etCombo['Spectal_Capacity'].combine_first(df_etCombo['Spectal_Progression'])

# Drop the 'Spectal_Progression' column
df_etCombo.drop('Spectal_Progression', axis=1, inplace=True)
df_etCombo.drop('Spectal_Capacity', axis=1, inplace=True)

##### Removing rows with no data. 

In [6]:
# Ensure 'id' is an integer
df_etCombo['id'] = df_etCombo['id'].astype(int)

# Trim spaces and adjust for case sensitivity in 'plotId'
df_etCombo['plotId'] = df_etCombo['plotId'].str.strip().str.lower()

# Filter out rows where 'id' is 1 and 'plotId' is 'nodata'
df_etCombo = df_etCombo[~((df_etCombo['id'] == 1) & (df_etCombo['plotId'] == 'no data'))]

##### Adding a column for user number. 

In [7]:
# Ensure that the "Valid_Name" column is treated as a string
df_etCombo['Valid_Name'] = df_etCombo['Valid_Name'].astype(str)

# Mapping from Valid_Name to assigned number
name_to_number = {
    '934367363': 7, 'andrewcr': 1, 'asdf': 7, 'cahillem': 2, 'janesk': 4,
    'Nicholas': 3, 'sawyerre': 5, 'shinzaas': 6, 'Stable plot seen in all data.': 7,
    'underhik': 7, 'nan': 7
}

# Default number for blank names
default_number = 7

# Function to get the number from Valid_Name
def get_number(name):
    if name.strip() == '':  # Check for blank names
        return default_number
    return name_to_number.get(name, default_number)

# Populate the 'user' column
df_etCombo['user'] = df_etCombo['Valid_Name'].apply(get_number)

##### Removing repeat observations that are due to data entry errors. 

In [8]:
# Identify duplicate rows based on 'user', 'tracker', and 'epoch'
duplicate_rows = df_etCombo[df_etCombo.duplicated(subset=['user', 'tracker', 'epoch'], keep='first')]

# Extract the 'tracker' numbers of the deleted rows
deleted_tracker_numbers = duplicate_rows['tracker'].unique()

# Count the number of duplicate rows
deleted_rows_count = len(duplicate_rows)

# Print the tracker numbers and the count of the deleted rows
print("Tracker numbers of the deleted rows:", deleted_tracker_numbers)
print("Count of the deleted rows:", deleted_rows_count)

# Now remove the duplicate rows, keeping only the first occurrence
df_etCombo = df_etCombo.drop_duplicates(subset=['user', 'tracker', 'epoch'], keep='first')

# Optional: Reset index after removing duplicates
df_etCombo.reset_index(drop=True, inplace=True)

Tracker numbers of the deleted rows: ['87950922' '81395819' '84152401' '66411237' '29398292' '97187302'
 '76577468' '18472705' '7581613' '10351455' '67290870' '40107720'
 '23179987' '58835211' '91186758' '96506508' '76927573' '69839659'
 '97765910' '19195268' '10265774' '19398624' '31920893' '65723809'
 '15723160' '43497672' '12493287' '48460786' '74866697' '45386448'
 '18389439' '58326896' '42212153' '39463480' '29895800' '28326592'
 '10586330' '33184069' '19048534' '40616112' '67226290' '95590134'
 '9642087' '21375171' '28828997' '17117791' '82492414' '42526660'
 '81626876' '66982560' '99655003' '36076168' '93051693' '91205046'
 '21431604' '59273721' '40266624' '6603721' '39237506' '54528681'
 '22351213' '24943628' '74702469' '69412785' '61499310' '48769782'
 '45782398' '49011078' '49975472' '30898710' '45904836' '28259348'
 '68946576' '60374549' '93089310' '20756865' '8515248' '41273984'
 '97216086' '16382155' '74126659' '25821022' '89864575' '43059704'
 '71752994' '22313675' '42237

#### Creating and populating the probability of loss/stable/growth columns. 

##### Get the number of epochs from our dataset. 

In [9]:
# Print the total number of rows in the DataFrame
total_rows = df_etCombo.shape[0]
print(f"Total number of rows in the DataFrame: {total_rows}")

# Extract unique numbers from the "epoch" column
epoch_list = df_etCombo["epoch"].unique().tolist()
print("Epoch List:", epoch_list)

# Ensure the reEval column is properly converted to integers for comparison
df_etCombo["reEval"] = pd.to_numeric(df_etCombo["reEval"], errors="coerce")

# Group by 'user' and 'tracker' and count the occurrences of each combination
user_tracker_counts = df_etCombo.groupby(["user", "tracker"]).size()

# Identify tracker numbers for combinations appearing more than 2 times
trackers_over_2 = user_tracker_counts[user_tracker_counts > 2].reset_index()
trackers_over_2_list = trackers_over_2['tracker'].unique()

# Print tracker numbers with more than 2 occurrences
if len(trackers_over_2_list) > 0:
    print(f"Tracker numbers with more than 2 occurrences: {trackers_over_2_list}")
else:
    print("No tracker numbers with more than 2 occurrences.")

# Count how many times each repetition occurred (e.g., 2 times, 3 times, etc.)
repetition_counts = user_tracker_counts.value_counts().sort_index()

# Print the results
print("Number of Times Unique User-Tracker Combos Repeated:")
for times, count in repetition_counts.items():
    print(f"{times} times: {count} combinations")

Total number of rows in the DataFrame: 2687
Epoch List: [1, 2, 3, 4, 5, 6, 7, 8, 9, 9999]
Tracker numbers with more than 2 occurrences: ['55400962' '94123684' '74866697']
Number of Times Unique User-Tracker Combos Repeated:
1 times: 2544 combinations
2 times: 67 combinations
3 times: 3 combinations


##### Add the necessary columns to eventually compress the unique user/point combinations into a single line. 

In [10]:
# Define the naming pattern for the new columns
months = [4, 6, 12]
conditions = ["Loss", "Stable", "Growth"]

# Generate column names based on the "epoch" numbers
new_columns = []
for epoch_num in epoch_list:
    for month in months:
        for condition in conditions:
            column_name = f"E{epoch_num}_Prob_{month}month_{condition}"
            new_columns.append(column_name)
            df_etCombo[column_name] = np.nan  # Initialize with NaN

print("Added Columns:", new_columns)

Added Columns: ['E1_Prob_4month_Loss', 'E1_Prob_4month_Stable', 'E1_Prob_4month_Growth', 'E1_Prob_6month_Loss', 'E1_Prob_6month_Stable', 'E1_Prob_6month_Growth', 'E1_Prob_12month_Loss', 'E1_Prob_12month_Stable', 'E1_Prob_12month_Growth', 'E2_Prob_4month_Loss', 'E2_Prob_4month_Stable', 'E2_Prob_4month_Growth', 'E2_Prob_6month_Loss', 'E2_Prob_6month_Stable', 'E2_Prob_6month_Growth', 'E2_Prob_12month_Loss', 'E2_Prob_12month_Stable', 'E2_Prob_12month_Growth', 'E3_Prob_4month_Loss', 'E3_Prob_4month_Stable', 'E3_Prob_4month_Growth', 'E3_Prob_6month_Loss', 'E3_Prob_6month_Stable', 'E3_Prob_6month_Growth', 'E3_Prob_12month_Loss', 'E3_Prob_12month_Stable', 'E3_Prob_12month_Growth', 'E4_Prob_4month_Loss', 'E4_Prob_4month_Stable', 'E4_Prob_4month_Growth', 'E4_Prob_6month_Loss', 'E4_Prob_6month_Stable', 'E4_Prob_6month_Growth', 'E4_Prob_12month_Loss', 'E4_Prob_12month_Stable', 'E4_Prob_12month_Growth', 'E5_Prob_4month_Loss', 'E5_Prob_4month_Stable', 'E5_Prob_4month_Growth', 'E5_Prob_6month_Loss', 

##### Populate the new columns based on the values in the tracker, user, epoch, reEval, and probability columns so that each unique user/tracker combo has only one row. 

In [11]:
# Helper function to extract JSON values and convert them to percentages
def extract_json_values(json_str):
    try:
        json_data = json.loads(json_str)
        return {
            "Loss": int(json_data.get("loss", "0%").replace("%", "")),
            "Stable": int(json_data.get("Stable", "0%").replace("%", "")),
            "Growth": int(json_data.get("Growth", "0%").replace("%", ""))
        }
    except (json.JSONDecodeError, ValueError, KeyError) as e:
        print(f"Error parsing JSON: {e}")
        return {"Loss": 0, "Stable": 0, "Growth": 0}

# Iterate through the DataFrame and apply logic based on 'reEval' column
for index, row in df_etCombo.iterrows():
    epoch_num = row["epoch"]  # Get the epoch number for the current row

    # If 'reEval' == 0, populate the correct epoch columns directly
    if row["reEval"] == 0:
        for month in [4, 6, 12]:
            for condition in ["Loss", "Stable", "Growth"]:
                column_name = f'E{epoch_num}_Prob_{month}month_{condition}'
                df_etCombo.at[index, column_name] = extract_json_values(
                    row[f"Probability_Forest_Loss_{month}_month"]
                )[condition]

    elif row["reEval"] == 1:
        # Search for the matching row with the same tracker, user, and 'reEval' == 0
        match = df_etCombo[
            (df_etCombo["tracker"] == row["tracker"]) &
            (df_etCombo["user"] == row["user"]) &
            (df_etCombo["reEval"] == 0)
        ]

        if not match.empty:
            # If a matching row is found, copy the values to the matched row
            match_index = match.index[0]  # Assume only one matching row

            for month in [4, 6, 12]:
                json_values = extract_json_values(row[f"Probability_Forest_Loss_{month}_month"])

                for condition, value in json_values.items():
                    target_column = f'E{epoch_num}_Prob_{month}month_{condition}'
                    df_etCombo.at[match_index, target_column] = value

            # Mark the current row for deletion
            df_etCombo.at[index, "delete_flag"] = True
        else:
            # If no matching row is found, populate the epoch-specific columns for this row
            for month in [4, 6, 12]:
                json_values = extract_json_values(row[f"Probability_Forest_Loss_{month}_month"])

                for condition, value in json_values.items():
                    target_column = f'E{epoch_num}_Prob_{month}month_{condition}'
                    df_etCombo.at[index, target_column] = value

# Delete rows with 'reEval' == 1 that had matching rows
df_etCombo = df_etCombo[df_etCombo["delete_flag"].isna()]

# Clean up by dropping the 'delete_flag' column
df_etCombo.drop(columns=["delete_flag"], inplace=True)

In [12]:
# Print the total number of rows in the DataFrame
total_rows = df_etCombo.shape[0]
print(f"Total number of rows in the DataFrame: {total_rows}")

Total number of rows in the DataFrame: 2617


##### Delete the unecessary columns. 

In [13]:
# Delete the specified columns from the DataFrame
columns_to_delete = [
    "Probability_Forest_Loss_4_month", 
    "Probability_Forest_Loss_6_month", 
    "Probability_Forest_Loss_12_month", 
    "epoch"
]

df_etCombo.drop(columns=columns_to_delete, inplace=True)

# Print confirmation
print(f"Deleted columns: {columns_to_delete}")

# Print all remaining column names as a list
print("Remaining columns:", df_etCombo.columns.tolist())

Deleted columns: ['Probability_Forest_Loss_4_month', 'Probability_Forest_Loss_6_month', 'Probability_Forest_Loss_12_month', 'epoch']
Remaining columns: ['id', 'plotId', 'Valid_Date', 'LT_YOD', 'Image_Condition', 'Spatial_Context', 'Valid_Name', 'Comment', 'User_IP', 'user_elapsed_time', 'tracker', 'reEval', 'Spectral_Capacity', 'user', 'E1_Prob_4month_Loss', 'E1_Prob_4month_Stable', 'E1_Prob_4month_Growth', 'E1_Prob_6month_Loss', 'E1_Prob_6month_Stable', 'E1_Prob_6month_Growth', 'E1_Prob_12month_Loss', 'E1_Prob_12month_Stable', 'E1_Prob_12month_Growth', 'E2_Prob_4month_Loss', 'E2_Prob_4month_Stable', 'E2_Prob_4month_Growth', 'E2_Prob_6month_Loss', 'E2_Prob_6month_Stable', 'E2_Prob_6month_Growth', 'E2_Prob_12month_Loss', 'E2_Prob_12month_Stable', 'E2_Prob_12month_Growth', 'E3_Prob_4month_Loss', 'E3_Prob_4month_Stable', 'E3_Prob_4month_Growth', 'E3_Prob_6month_Loss', 'E3_Prob_6month_Stable', 'E3_Prob_6month_Growth', 'E3_Prob_12month_Loss', 'E3_Prob_12month_Stable', 'E3_Prob_12month_Growt

##### Generating and populating columns for the average probability from that user. 

In [14]:
# Define the column types we need to create average columns for
column_types = [
    'Prob_4month_Loss', 'Prob_4month_Stable', 'Prob_4month_Growth',
    'Prob_6month_Loss', 'Prob_6month_Stable', 'Prob_6month_Growth',
    'Prob_12month_Loss', 'Prob_12month_Stable', 'Prob_12month_Growth'
]

# Track the columns to be added to the DataFrame
new_columns_dict = {}

# Ensure every epoch number from 'epoch_list' is used for each column type
for col_type in column_types:
    column_list = []

    # Collect all columns for the current column type using the epoch list
    for epoch in epoch_list:
        col_name = f'E{epoch}_{col_type}'  # Dynamically create column name

        # Check if the column exists in the DataFrame
        if col_name in df_etCombo.columns:
            print(f"Found column: {col_name}")
            column_list.append(col_name)
        else:
            print(f"Missing column: {col_name}")

    # Calculate the average across the selected columns
    if column_list:
        new_column_name = f'Ave_{col_type}'
        new_columns_dict[new_column_name] = df_etCombo[column_list].mean(axis=1)

        print(f"Added column '{new_column_name}' using columns: {column_list}")
    else:
        print(f"No valid columns found for {col_type}")

# Concatenate the new columns to the original DataFrame in one go
df_etCombo = pd.concat([df_etCombo, pd.DataFrame(new_columns_dict)], axis=1)

# Print the list of all added average columns
print("\nAdded average columns:", list(new_columns_dict.keys()))

Found column: E1_Prob_4month_Loss
Found column: E2_Prob_4month_Loss
Found column: E3_Prob_4month_Loss
Found column: E4_Prob_4month_Loss
Found column: E5_Prob_4month_Loss
Found column: E6_Prob_4month_Loss
Found column: E7_Prob_4month_Loss
Found column: E8_Prob_4month_Loss
Found column: E9_Prob_4month_Loss
Found column: E9999_Prob_4month_Loss
Added column 'Ave_Prob_4month_Loss' using columns: ['E1_Prob_4month_Loss', 'E2_Prob_4month_Loss', 'E3_Prob_4month_Loss', 'E4_Prob_4month_Loss', 'E5_Prob_4month_Loss', 'E6_Prob_4month_Loss', 'E7_Prob_4month_Loss', 'E8_Prob_4month_Loss', 'E9_Prob_4month_Loss', 'E9999_Prob_4month_Loss']
Found column: E1_Prob_4month_Stable
Found column: E2_Prob_4month_Stable
Found column: E3_Prob_4month_Stable
Found column: E4_Prob_4month_Stable
Found column: E5_Prob_4month_Stable
Found column: E6_Prob_4month_Stable
Found column: E7_Prob_4month_Stable
Found column: E8_Prob_4month_Stable
Found column: E9_Prob_4month_Stable
Found column: E9999_Prob_4month_Stable
Added col

##### Generating and populating columns for the most recent observation by that user. 

In [15]:
# Track the new columns to be added
new_columns_dict = {}

# Iterate over each column type and create the new "Rec_" columns
for col_type in column_types:
    rec_column = f'Rec_{col_type}'
    new_columns_dict[rec_column] = pd.Series([None] * len(df_etCombo))  # Initialize with None

    # Iterate over epoch numbers in reverse order to prioritize higher epochs (e.g., E999 first)
    for epoch in sorted(epoch_list, reverse=True):
        column_name = f'E{epoch}_{col_type}'

        # Only attempt to combine if the column exists in the DataFrame
        if column_name in df_etCombo.columns:
            # Use combine_first to prioritize non-None values from existing columns
            new_columns_dict[rec_column] = new_columns_dict[rec_column].combine_first(df_etCombo[column_name])

# Add all the new "Rec_" columns to the DataFrame at once using pd.concat()
df_etCombo = pd.concat([df_etCombo, pd.DataFrame(new_columns_dict)], axis=1)

# Print the added columns
added_columns = list(new_columns_dict.keys())
print("\nAdded columns:", added_columns)


Added columns: ['Rec_Prob_4month_Loss', 'Rec_Prob_4month_Stable', 'Rec_Prob_4month_Growth', 'Rec_Prob_6month_Loss', 'Rec_Prob_6month_Stable', 'Rec_Prob_6month_Growth', 'Rec_Prob_12month_Loss', 'Rec_Prob_12month_Stable', 'Rec_Prob_12month_Growth']


##### Creating a 0/1 column for when Loss probability is highest. 
    This will be treated as your typical loss/no loss interpreter observations that has no associated uncertainty. 

In [16]:
# Define a pattern to find all month types dynamically
pattern = re.compile(r'Prob_(\d+)month_(Loss|Stable|Growth)')

# Extract all unique month durations from column names
month_durations = sorted({match.group(1) for col in df_etCombo.columns for match in [pattern.search(col)] if match})

# Track the new columns to be added to the DataFrame
new_columns_dict = {}

# Create the new TF columns using epoch_list + ['Ave', 'Rec']
for prefix in [f'E{epoch}' for epoch in epoch_list] + ['Ave', 'Rec']:
    for month in month_durations:
        # Define the column names for loss, stable, growth, and TF
        loss_col = f'{prefix}_Prob_{month}month_Loss'
        stable_col = f'{prefix}_Prob_{month}month_Stable'
        growth_col = f'{prefix}_Prob_{month}month_Growth'
        tf_col = f'{prefix}_TF_{month}month_Loss'

        # Check if all required columns exist
        if all(col in df_etCombo.columns for col in [loss_col, stable_col, growth_col]):
            # Compute the TF column values, handling NaN values properly
            tf_values = np.where(
                df_etCombo[[loss_col, stable_col, growth_col]].isna().any(axis=1),
                np.nan,  # If any of the values is NaN, set TF to NaN
                np.where(
                    (df_etCombo[loss_col] > df_etCombo[stable_col]) &
                    (df_etCombo[loss_col] > df_etCombo[growth_col]),
                    1,  # Set TF to 1 if loss is the highest
                    0   # Otherwise, set TF to 0
                )
            )

            # Store the TF column values in the dictionary
            new_columns_dict[tf_col] = pd.Series(tf_values, index=df_etCombo.index)

# Add all the new TF columns to the DataFrame at once using pd.concat()
new_tf_df = pd.DataFrame(new_columns_dict, index=df_etCombo.index)

# Add the new TF columns to the original DataFrame without creating extra rows
df_etCombo = pd.concat([df_etCombo, new_tf_df], axis=1)

# Print the added TF columns
added_tf_columns = list(new_columns_dict.keys())
print("\nAdded TF columns:", added_tf_columns)

# Drop rows that only have NaN in the 'tracker' column
df_etCombo.dropna(subset=['tracker'], inplace=True)


Added TF columns: ['E1_TF_12month_Loss', 'E1_TF_4month_Loss', 'E1_TF_6month_Loss', 'E2_TF_12month_Loss', 'E2_TF_4month_Loss', 'E2_TF_6month_Loss', 'E3_TF_12month_Loss', 'E3_TF_4month_Loss', 'E3_TF_6month_Loss', 'E4_TF_12month_Loss', 'E4_TF_4month_Loss', 'E4_TF_6month_Loss', 'E5_TF_12month_Loss', 'E5_TF_4month_Loss', 'E5_TF_6month_Loss', 'E6_TF_12month_Loss', 'E6_TF_4month_Loss', 'E6_TF_6month_Loss', 'E7_TF_12month_Loss', 'E7_TF_4month_Loss', 'E7_TF_6month_Loss', 'E8_TF_12month_Loss', 'E8_TF_4month_Loss', 'E8_TF_6month_Loss', 'E9_TF_12month_Loss', 'E9_TF_4month_Loss', 'E9_TF_6month_Loss', 'E9999_TF_12month_Loss', 'E9999_TF_4month_Loss', 'E9999_TF_6month_Loss', 'Ave_TF_12month_Loss', 'Ave_TF_4month_Loss', 'Ave_TF_6month_Loss', 'Rec_TF_12month_Loss', 'Rec_TF_4month_Loss', 'Rec_TF_6month_Loss']


#### Attaching point metadata from Polygon Table to the Event Table. 

In [17]:
# Load the polygon table CSV into a DataFrame
df_polygon = pd.read_csv('polygonTable_combined.csv')

# Ensure the 'tracker' column is of the same type in both DataFrames
df_etCombo['tracker'] = df_etCombo['tracker'].astype(str)
df_polygon['tracker'] = df_polygon['tracker'].astype(str)

# Keeping only the first entry for each 'tracker'
df_polygon = df_polygon.drop_duplicates('tracker', keep='first')

# Merging again
df_etCombo = df_etCombo.merge(df_polygon[['tracker', 'geo', 'json']], on='tracker', how='left')

##### Separating out longitude and latitude for GEE upload. 

In [18]:
# Function to extract longitude and latitude from the 'geo' column
def extract_coordinates(geo_json):
    try:
        # Ensure the input is a string; if not, convert it to a string
        if isinstance(geo_json, str):
            # Replace single quotes with double quotes to ensure valid JSON
            geo_json = geo_json.replace("'", '"')

        # Parse the JSON string
        geo_dict = json.loads(geo_json)

        # Extract the coordinates assuming the correct structure
        coords = geo_dict.get('coordinates', [None, None])
        return coords[0], coords[1]

    except (json.JSONDecodeError, KeyError, TypeError, ValueError) as e:
        print(f"Error parsing JSON: {e}")
        return None, None

# Apply the function to extract coordinates and create new columns
df_etCombo[['longitude', 'latitude']] = pd.DataFrame(
    df_etCombo['geo'].apply(extract_coordinates).tolist(), index=df_etCombo.index
)

# Change the data type of 'longitude' and 'latitude' to float, handling NaN values
df_etCombo['longitude'] = pd.to_numeric(df_etCombo['longitude'], errors='coerce')
df_etCombo['latitude'] = pd.to_numeric(df_etCombo['latitude'], errors='coerce')

# Drop the 'geo' column
df_etCombo.drop(columns=['geo'], inplace=True)

# Confirm the output
print(df_etCombo[['longitude', 'latitude']].head())

    longitude   latitude
0  103.841873  12.127876
1  107.085125  12.983627
2  104.668876  13.678627
3  105.789876  11.563377
4  104.229873  14.220876


##### Getting the stratum ID for checking accuracy later. 

In [19]:
# Renaming "json" column to avoid confusion. 
df_etCombo.rename(columns={'json': 'pt_metadata'}, inplace=True)

# Function to extract stratumID
def extract_stratumID(json_str):
    try:
        # Load the string as JSON
        data = json.loads(json_str)
        # Extract stratumID from the last entry in the list
        stratumID = data[-1]['stratumID'] if data else None
    except (json.JSONDecodeError, IndexError, KeyError):
        stratumID = None
    return stratumID

# Apply the function to each row in the 'pt_metadata' column
df_etCombo['stratumIdIntPts'] = df_etCombo['pt_metadata'].apply(extract_stratumID)

# Drop the 'geo' column
df_etCombo.drop(columns=['pt_metadata'], inplace=True)

#### Reorganizing the columns. 

In [20]:
# Step 1: Define the fixed columns that should be at the start and end
start_columns = ['tracker', 'longitude', 'latitude', 'stratumIdIntPts', 'reEval', 'user']
end_columns = [
    'Image_Condition', 'Spatial_Context', 'Spectral_Capacity', 'id', 'plotId',
    'Valid_Name', 'Comment', 'User_IP', 'user_elapsed_time', 'Valid_Date', 'LT_YOD'
]

# Step 2: Extract all columns and categorize them
all_columns = df_etCombo.columns

# Helper function to safely extract numeric parts from column names
def extract_epoch_number(col):
    try:
        # Extract the number part from columns like 'E999_Prob_4month_Loss'
        return int(re.search(r'E(\d+)', col).group(1))
    except (AttributeError, ValueError):
        return float('inf')  # Place non-matching columns at the end

def extract_month_number(col):
    try:
        # Extract the month number (e.g., '4' from 'Prob_4month_Loss')
        return int(re.search(r'(\d+)month', col).group(1))
    except (AttributeError, ValueError):
        return float('inf')  # Place non-matching columns at the end

# Separate columns into REC, AVE, and Epoch columns
rec_columns = sorted(
    [col for col in all_columns if col.startswith('Rec_')],
    key=lambda x: (extract_epoch_number(x), x)
)
ave_columns = sorted(
    [col for col in all_columns if col.startswith('Ave_')],
    key=lambda x: (extract_epoch_number(x), x)
)
epoch_columns = sorted(
    [col for col in all_columns if col.startswith('E') and ('_Prob_' in col or '_TF_' in col)],
    key=lambda x: (extract_epoch_number(x), extract_month_number(x), x)
)

# Step 3: Combine columns in the desired order
new_column_order = start_columns + rec_columns + ave_columns + epoch_columns + end_columns

# Step 4: Ensure no columns are left out or duplicated
missing_columns = set(all_columns) - set(new_column_order)
if missing_columns:
    print("Warning: The following columns were not included in the new order:", missing_columns)
    new_column_order.extend(missing_columns)

# Step 5: Reorder the DataFrame columns
df_etCombo = df_etCombo[new_column_order]

print(list(df_etCombo))


['tracker', 'longitude', 'latitude', 'stratumIdIntPts', 'reEval', 'user', 'Rec_Prob_12month_Growth', 'Rec_Prob_12month_Loss', 'Rec_Prob_12month_Stable', 'Rec_Prob_4month_Growth', 'Rec_Prob_4month_Loss', 'Rec_Prob_4month_Stable', 'Rec_Prob_6month_Growth', 'Rec_Prob_6month_Loss', 'Rec_Prob_6month_Stable', 'Rec_TF_12month_Loss', 'Rec_TF_4month_Loss', 'Rec_TF_6month_Loss', 'Ave_Prob_12month_Growth', 'Ave_Prob_12month_Loss', 'Ave_Prob_12month_Stable', 'Ave_Prob_4month_Growth', 'Ave_Prob_4month_Loss', 'Ave_Prob_4month_Stable', 'Ave_Prob_6month_Growth', 'Ave_Prob_6month_Loss', 'Ave_Prob_6month_Stable', 'Ave_TF_12month_Loss', 'Ave_TF_4month_Loss', 'Ave_TF_6month_Loss', 'E1_Prob_4month_Growth', 'E1_Prob_4month_Loss', 'E1_Prob_4month_Stable', 'E1_TF_4month_Loss', 'E1_Prob_6month_Growth', 'E1_Prob_6month_Loss', 'E1_Prob_6month_Stable', 'E1_TF_6month_Loss', 'E1_Prob_12month_Growth', 'E1_Prob_12month_Loss', 'E1_Prob_12month_Stable', 'E1_TF_12month_Loss', 'E2_Prob_4month_Growth', 'E2_Prob_4month_Los

#### Making sure everything is in integer format for RF classification. 

In [21]:
# Build regex pattern to match endings like '4month_Loss', '12month_Stable', etc.
pattern = re.compile(r'\d*month_(Loss|Stable|Growth)$')

# Filter columns matching the pattern
allDependentColumns = [col for col in df_etCombo.columns if pattern.search(col)]

# Print the matching columns
print("Columns matching the specified endings:", allDependentColumns)

# List to store columns that are numeric but not integer
non_integer_numeric_cols = []

for col in allDependentColumns:
    if pd.api.types.is_numeric_dtype(df_etCombo[col]):
        if not np.all(df_etCombo[col].dropna() == df_etCombo[col].dropna().astype(int)):
            non_integer_numeric_cols.append(col)

print("Non-integer numeric columns:", non_integer_numeric_cols)

Columns matching the specified endings: ['Rec_Prob_12month_Growth', 'Rec_Prob_12month_Loss', 'Rec_Prob_12month_Stable', 'Rec_Prob_4month_Growth', 'Rec_Prob_4month_Loss', 'Rec_Prob_4month_Stable', 'Rec_Prob_6month_Growth', 'Rec_Prob_6month_Loss', 'Rec_Prob_6month_Stable', 'Rec_TF_12month_Loss', 'Rec_TF_4month_Loss', 'Rec_TF_6month_Loss', 'Ave_Prob_12month_Growth', 'Ave_Prob_12month_Loss', 'Ave_Prob_12month_Stable', 'Ave_Prob_4month_Growth', 'Ave_Prob_4month_Loss', 'Ave_Prob_4month_Stable', 'Ave_Prob_6month_Growth', 'Ave_Prob_6month_Loss', 'Ave_Prob_6month_Stable', 'Ave_TF_12month_Loss', 'Ave_TF_4month_Loss', 'Ave_TF_6month_Loss', 'E1_Prob_4month_Growth', 'E1_Prob_4month_Loss', 'E1_Prob_4month_Stable', 'E1_TF_4month_Loss', 'E1_Prob_6month_Growth', 'E1_Prob_6month_Loss', 'E1_Prob_6month_Stable', 'E1_TF_6month_Loss', 'E1_Prob_12month_Growth', 'E1_Prob_12month_Loss', 'E1_Prob_12month_Stable', 'E1_TF_12month_Loss', 'E2_Prob_4month_Growth', 'E2_Prob_4month_Loss', 'E2_Prob_4month_Stable', 'E2_

In [22]:
for col in non_integer_numeric_cols:
    df_etCombo[col] = df_etCombo[col].round().astype(int)

#### Exporting the full CSV (user observations still separate for a single point).

In [23]:
# EXPORTING!!!!!

# Save the DataFrame to a new CSV file
df_etCombo.to_csv('camIntObservationsFull.csv', index=False)

#### Formatting for GEE.  

In [24]:
# Export GEE CSV with separate users for repeated points. 

# EXPORTING!!!!!

#Create the new DataFrame with selected columns
df_etComboGeeSeparate = df_etCombo[['tracker', 'latitude', 'longitude', 'user', 'Rec_Prob_12month_Loss', 'Rec_TF_12month_Loss', 'Ave_Prob_12month_Loss', 'Ave_TF_12month_Loss']].copy()

print(list(df_etComboGeeSeparate))

# Save the DataFrame to a new CSV file
df_etComboGeeSeparate.to_csv('camIntObservationsGee_SeparateUsers.csv', index=False)

['tracker', 'latitude', 'longitude', 'user', 'Rec_Prob_12month_Loss', 'Rec_TF_12month_Loss', 'Ave_Prob_12month_Loss', 'Ave_TF_12month_Loss']


In [25]:
#Create the new DataFrame with selected columns
df_etComboGee = df_etCombo[['tracker', 'latitude', 'longitude', 'user']].copy()

# Drop the user column since we are changing the format for the other GEE file. 
df_etComboGee = df_etComboGee.drop('user', axis=1)

print("Created a new DataFrame called 'df_etComboGee'.")

Created a new DataFrame called 'df_etComboGee'.


In [26]:
# Add new columns needed for GEE. 

# Add 'user1' and 'user2' columns to the new DataFrame
df_etComboGee['user1'] = np.nan
df_etComboGee['user2'] = np.nan

# Define the month and condition options
months = [4, 6, 12]
conditions = ['Loss', 'Stable', 'Growth']

# Generate all combinations for the new columns and add them to the DataFrame
new_columns = [f'Rec_AveProb_{month}month_{condition}' for month in months for condition in conditions]

# Add these columns to the DataFrame with NaN values
for col in new_columns:
    df_etComboGee[col] = np.nan

# Print the list of columns in the new DataFrame
print("\nColumns in 'df_etComboGee':")
print(df_etComboGee.columns.tolist())

# Count the total number of rows in the DataFrame
total_rows = df_etComboGee.shape[0]

# Print the result
print(f"Total number of rows: {total_rows}")


Columns in 'df_etComboGee':
['tracker', 'latitude', 'longitude', 'user1', 'user2', 'Rec_AveProb_4month_Loss', 'Rec_AveProb_4month_Stable', 'Rec_AveProb_4month_Growth', 'Rec_AveProb_6month_Loss', 'Rec_AveProb_6month_Stable', 'Rec_AveProb_6month_Growth', 'Rec_AveProb_12month_Loss', 'Rec_AveProb_12month_Stable', 'Rec_AveProb_12month_Growth']
Total number of rows: 2617


In [27]:
# Remove rows with duplicate 'tracker' values, keeping only the first occurrence
df_etComboGee = df_etComboGee.drop_duplicates(subset=['tracker'], keep='first')

# Print confirmation
print("Removed rows with duplicate 'tracker' values from 'df_etComboGee'.")
print(f"New shape of 'df_etComboGee': {df_etComboGee.shape}")

Removed rows with duplicate 'tracker' values from 'df_etComboGee'.
New shape of 'df_etComboGee': (2031, 14)


##### Combining all observations for each point for training. (Some points have multiple interpreters look at them which will not work for training). 

In [28]:
# Filling the 'user1' and 'user2' columns. 
# First, initialize user1 and user2 columns with NaN values
df_etComboGee['user1'] = np.nan
df_etComboGee['user2'] = np.nan

# Iterate over the new DataFrame to fill the user1 and user2 columns
for index, row in df_etComboGee.iterrows():
    tracker_value = row['tracker']

    # Find all matching rows in the original DataFrame based on the tracker value
    matching_rows = df_etCombo[df_etCombo['tracker'] == tracker_value]

    # Extract user numbers from matching rows
    user_values = matching_rows['user'].unique()

    # Fill the user1 and user2 columns if available
    if len(user_values) >= 1:
        df_etComboGee.at[index, 'user1'] = user_values[0]
    if len(user_values) >= 2:
        df_etComboGee.at[index, 'user2'] = user_values[1]

    # Print tracker and user values if there are more than 2 matching users
    if len(user_values) > 2:
        print(f"Tracker {tracker_value} has more than two users:")
        for i, user in enumerate(user_values[2:], start=3):
            print(f"User {i}: {user}")

# Print confirmation of completion
print("\nCompleted filling user1 and user2 columns.")
print(df_etComboGee.head())


Completed filling user1 and user2 columns.
    tracker   latitude   longitude  user1  user2  Rec_AveProb_4month_Loss  \
0  87950922  12.127876  103.841873    1.0    2.0                      NaN   
1  81395819  12.983627  107.085125    1.0    3.0                      NaN   
2  84152401  13.678627  104.668876    1.0    4.0                      NaN   
3  66411237  11.563377  105.789876    1.0    5.0                      NaN   
4  29398292  14.220876  104.229873    1.0    6.0                      NaN   

   Rec_AveProb_4month_Stable  Rec_AveProb_4month_Growth  \
0                        NaN                        NaN   
1                        NaN                        NaN   
2                        NaN                        NaN   
3                        NaN                        NaN   
4                        NaN                        NaN   

   Rec_AveProb_6month_Loss  Rec_AveProb_6month_Stable  \
0                      NaN                        NaN   
1                      N

In [29]:
# Count the total number of rows in the DataFrame
total_rows = df_etComboGee.shape[0]

# Count the total number of unique tracker numbers
unique_trackers = df_etComboGee['tracker'].nunique()

# Count the number of rows where 'user2' is not NaN
user2_count = df_etComboGee['user2'].notna().sum()

# Print the result
print(f"Total number of rows: {total_rows}")
print(f"Total number of unique tracker numbers: {unique_trackers}")
print(f"Number of rows with a value in the 'user2' column: {user2_count}")

Total number of rows: 2031
Total number of unique tracker numbers: 2031
Number of rows with a value in the 'user2' column: 583


In [30]:
# Iterate through all Rec_AveProb_<MM>month_<condition> columns in the new DataFrame
for month in months:
    for condition in conditions:
        # Define the corresponding column name in the new DataFrame
        new_col_name = f'Rec_AveProb_{month}month_{condition}'
        
        # Define the corresponding column name in the original DataFrame
        orig_col_name = f'Rec_Prob_{month}month_{condition}'

        # Ensure the original column exists before proceeding
        if orig_col_name in df_etCombo.columns:
            # Iterate over each row in the new DataFrame
            for index, row in df_etComboGee.iterrows():
                tracker_value = row['tracker']

                # Find matching rows in the original DataFrame based on the tracker number
                matching_rows = df_etCombo[df_etCombo['tracker'] == tracker_value]

                if not matching_rows.empty:
                    # Calculate the average value of the matching rows for the corresponding column
                    avg_value = matching_rows[orig_col_name].mean()

                    # Assign the average value to the new DataFrame
                    df_etComboGee.at[index, new_col_name] = avg_value

# Print confirmation of completion
print("\nCompleted filling Rec_AveProb_<MM>month_<condition> columns.")
print(df_etComboGee.head())


Completed filling Rec_AveProb_<MM>month_<condition> columns.
    tracker   latitude   longitude  user1  user2  Rec_AveProb_4month_Loss  \
0  87950922  12.127876  103.841873    1.0    2.0                      0.0   
1  81395819  12.983627  107.085125    1.0    3.0                     75.0   
2  84152401  13.678627  104.668876    1.0    4.0                      2.5   
3  66411237  11.563377  105.789876    1.0    5.0                      0.0   
4  29398292  14.220876  104.229873    1.0    6.0                      2.5   

   Rec_AveProb_4month_Stable  Rec_AveProb_4month_Growth  \
0                      100.0                        0.0   
1                       25.0                        0.0   
2                       97.5                        0.0   
3                      100.0                        0.0   
4                       97.5                        0.0   

   Rec_AveProb_6month_Loss  Rec_AveProb_6month_Stable  \
0                      0.0                      100.0   
1     

##### Create TF columns for the new combined probability values. 

In [31]:
# Iterate over all combinations of month and condition
for month in months:
    for condition in conditions:
        # Define the corresponding AveProb column name
        prob_col = f'Rec_AveProb_{month}month_{condition}'

        # Define the new AveTF column name
        tf_col = f'Rec_AveTF_{month}month_{condition}'

        # Ensure the AveProb column exists in the DataFrame
        if prob_col in df_etComboGee.columns:
            # Create the new AveTF column based on the value in the AveProb column
            df_etComboGee[tf_col] = np.where(
                df_etComboGee[prob_col] > 50.0, 1, 0
            )

# Print confirmation of completion
print("\nAdded and populated Rec_AveTF_<MM>month_<condition> columns.")
print(df_etComboGee.head())


Added and populated Rec_AveTF_<MM>month_<condition> columns.
    tracker   latitude   longitude  user1  user2  Rec_AveProb_4month_Loss  \
0  87950922  12.127876  103.841873    1.0    2.0                      0.0   
1  81395819  12.983627  107.085125    1.0    3.0                     75.0   
2  84152401  13.678627  104.668876    1.0    4.0                      2.5   
3  66411237  11.563377  105.789876    1.0    5.0                      0.0   
4  29398292  14.220876  104.229873    1.0    6.0                      2.5   

   Rec_AveProb_4month_Stable  Rec_AveProb_4month_Growth  \
0                      100.0                        0.0   
1                       25.0                        0.0   
2                       97.5                        0.0   
3                      100.0                        0.0   
4                       97.5                        0.0   

   Rec_AveProb_6month_Loss  Rec_AveProb_6month_Stable  ...  \
0                      0.0                      100.0  ...

##### Export the dataframe for GEE. 

In [32]:
print(list(df_etComboGee))

['tracker', 'latitude', 'longitude', 'user1', 'user2', 'Rec_AveProb_4month_Loss', 'Rec_AveProb_4month_Stable', 'Rec_AveProb_4month_Growth', 'Rec_AveProb_6month_Loss', 'Rec_AveProb_6month_Stable', 'Rec_AveProb_6month_Growth', 'Rec_AveProb_12month_Loss', 'Rec_AveProb_12month_Stable', 'Rec_AveProb_12month_Growth', 'Rec_AveTF_4month_Loss', 'Rec_AveTF_4month_Stable', 'Rec_AveTF_4month_Growth', 'Rec_AveTF_6month_Loss', 'Rec_AveTF_6month_Stable', 'Rec_AveTF_6month_Growth', 'Rec_AveTF_12month_Loss', 'Rec_AveTF_12month_Stable', 'Rec_AveTF_12month_Growth']


In [33]:
# Save the DataFrame to a new CSV file
df_etComboGee.to_csv('camIntObservationsGee_CombinedByTracker.csv', index=False)